# import libraries

In [1]:
import mediapipe as mp
import cv2 
import time 
import pyautogui
import numpy as np

# Hand Detector and Button Class


In [2]:
class HandDetector:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.mp_drawing = mp.solutions.drawing_utils
        self.hands = self.mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    def detect_hands(self, frame, draw=True):
        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(rgb_img)
        if self.results.multi_hand_landmarks:
            for hand_landmarks in self.results.multi_hand_landmarks:
                if draw:
                    self.mp_drawing.draw_landmarks(
                        frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                        self.mp_drawing.DrawingSpec(color=(245, 120, 66), thickness=2, circle_radius=2),
                        self.mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                    )

        return frame

    def find_hand_positions(self, frame, handNo=0, draw=True):
        lm_list = []
        if self.results.multi_hand_landmarks:
            my_hand = self.results.multi_hand_landmarks[handNo]
            for Id, lm in enumerate(my_hand.landmark):
                h, w, c = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append([Id, cx, cy])
                if draw:
                    cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        return lm_list

    def detect_fingers(self, frame):
        fingers_up = [0, 0, 0, 0, 0]  # Thumb, Index, Middle, Ring, Little
        if self.results.multi_hand_landmarks:
            hand_landmarks = self.results.multi_hand_landmarks[0]

            # Debugging: Print thumb tip and base coordinates
            thumb_tip = hand_landmarks.landmark[4]
            thumb_base = hand_landmarks.landmark[5]
            

            # Thumb (Based on the position of thumb tip and base)
            if thumb_tip.y <= thumb_base.y:  # If thumb tip is above thumb base
                fingers_up[0] = 1

            # Other fingers (Based on the position of finger tip and base)
            for finger_id in range(1, 5):
                finger_tip = hand_landmarks.landmark[finger_id * 4 + 4]
                finger_base = hand_landmarks.landmark[finger_id * 4 +1]
                
                if finger_tip.y < finger_base.y:  # If finger tip is above finger base
                    fingers_up[finger_id] = 1

        return fingers_up




In [3]:
class Button():
    def __init__(self, pos, text, size=[50,50]):
        self.pos = pos 
        self.size = size
        self.text = text

In [4]:
keys = [["Q","W","E","R","T","Y","U","I","O","P"]
       ,["A","S","D","F","G","H","J","K","L",";"]
       ,["Z","X","C","V","B","N","M",",",".","/"]]

In [5]:
#drawing the buttons 

def draw_buttons(img, buttons):
    for button in buttons:
        x, y = button.pos
        w, h = button.size
        cv2.rectangle(img, button.pos, (x + w, y + h), (255, 0, 255), cv2.FILLED)
        cv2.putText(img, button.text, (x + 14, y + 36), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
    return img

In [6]:
buttons = []

for i in range(len(keys)):
    for j, key in enumerate(keys[i]):
        buttons.append(Button([60*j+25, 60*i+25], key))

# Hand Gesture Recognition and Virtual Keyboard Control

In [ ]:
detector = HandDetector()
cap = cv2.VideoCapture(0)
final_text =""

while cap.isOpened():
    
    ret, frame = cap.read()
    
    frame = detector.detect_hands(frame)
    
    frame = draw_buttons(frame, buttons) 
    lm_list = detector.find_hand_positions(frame, draw=False)
    fingers = detector.detect_fingers(frame)
    
    if lm_list:
        for button in buttons:
            x, y = button.pos
            w, h = button.size
            if x < lm_list[8][1] < x + w and y<lm_list[8][2]<y+h:
                cv2.rectangle(frame, button.pos, (x + w, y + h), (175, 0, 175), cv2.FILLED)
                cv2.putText(frame, button.text, (x + 14, y + 36), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
                
                if fingers[1] == 1 and fingers[2] == 1:
                    cv2.rectangle(frame, button.pos, (x + w, y + h), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, button.text, (x + 14, y + 36), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
                    final_text +=button.text
                    time.sleep(0.5)
                    
    cv2.rectangle(frame, (50,350), (600,450), (175, 0, 175), cv2.FILLED)
    cv2.putText(frame, final_text, (60,425), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
    # Display
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Hand Gesture Recognition and Virtual Mouse Control

In [ ]:
preTime = 0
curTime = 0
detector = HandDetector()

# Screen resolution
wScreen, hScreen = pyautogui.size()
# Webcam resolution
wCam, hCam = 1028, 840

cap = cv2.VideoCapture(0)

while cap.isOpened():
    
    ret, frame = cap.read()
    
    frame = detector.detect_hands(frame)
    
    # Get the tip of the index
    m_list = detector.find_hand_positions(frame, handNo=0, draw=False)
    
    if len(m_list) != 0:
        x1, y1 = m_list[8][1:]  # Index finger tip
        
        # Check which fingers are up
        fingers = detector.detect_fingers(frame)
        
        # Move the cursor
        if fingers[1] == 1 and fingers[2] == 0:
            # Interpolate the x-coordinate to screen resolution
            x3 = np.interp(x1, (0, wCam), (0, wScreen))
            # Interpolate the y-coordinate to screen resolution
            y3 = np.interp(y1, (0, hCam), (0, hScreen))
            
            pyautogui.moveTo(wScreen-x3, y3)
            cv2.circle(frame,(x1,y1),15,(255,0,255),cv2.FILLED)
        # click     
        if fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 0:
            pyautogui.click(clicks=1)
            time.sleep(0.15)
            
        #scroll    
        if fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 1:
            pyautogui.scroll(10)
            
    # FPS rate
    curTime = time.time()
    fps = 1 / (curTime - preTime)
    preTime = curTime

    cv2.putText(frame, f"FPS: {int(fps)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (245, 117, 66), 2, cv2.LINE_AA)
    
    # Display
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
